<a href="https://colab.research.google.com/github/cartejaune/stock-screener/blob/main/prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet
!pip install requests
# de l'article https://medium.com/bitgrit-data-science-publication/ethereum-price-prediction-with-python-3b3805e6e512

In [32]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [48]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-16,"$3,180.45","$3,181.62","$3,055.17","$3,127.83","$3,127.83",12352406833
2022-02-17,"$3,126.86","$3,154.62","$2,861.85","$2,881.48","$2,881.48",15860206214
2022-02-18,"$2,884.34","$2,937.31","$2,761.64","$2,785.73","$2,785.73",15748173433
2022-02-19,"$2,784.87","$2,826.35","$2,707.38","$2,763.70","$2,763.70",9774183169
2022-02-20,"$2,763.16","$2,763.16","$2,611.79","$2,621.80","$2,621.80",11663598592


In [49]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1565 entries, 2017-11-09 to 2022-02-20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1565 non-null   float64
 1   High       1565 non-null   float64
 2   Low        1565 non-null   float64
 3   Close      1565 non-null   float64
 4   Adj Close  1565 non-null   float64
 5   Volume     1565 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 85.6 KB


In [50]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [51]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [52]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [53]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [54]:
df.tail()

,ds,y
1560,2022-02-16,"$3,180.45"
1561,2022-02-17,"$3,126.86"
1562,2022-02-18,"$2,884.34"
1563,2022-02-19,"$2,784.87"
1564,2022-02-20,"$2,763.16"


In [55]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [56]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [57]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
1925,2023-02-16
1926,2023-02-17
1927,2023-02-18
1928,2023-02-19
1929,2023-02-20


In [58]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1925,2023-02-16,"$3,230.19","$1,004.98","$5,423.95"
1926,2023-02-17,"$3,206.94","$1,004.95","$5,347.15"
1927,2023-02-18,"$3,164.08",$941.13,"$5,292.92"
1928,2023-02-19,"$3,155.46",$954.95,"$5,255.76"
1929,2023-02-20,"$3,149.45",$978.81,"$5,291.29"


In [62]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
ajourd = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')

a=forecast[forecast['ds'] == ajourd]['yhat'].item()
b=forecast[forecast['ds'] == next_day]['yhat'].item()
print(a-b)
#si positif demain chute du cours si négatif demain hausse du cours

71.16054038812445


In [63]:
plot_plotly(m, forecast)

In [47]:
plot_components_plotly(m, forecast)